# Парсинг HTML. BeautifulSoup, MongoDB

In [1]:
from bs4 import BeautifulSoup as bs
import requests
import re
import pandas as pd
from pprint import pprint

In [9]:
def seach_hh(vacancy , end_page = 1):

    vacancy_list = []
    
    params = {
        'text': vacancy, \
        'search_field': 'name', \
        'items_on_page': '100', \
        'page': ''
    }
    
    headers = {
        'User-Agent': 'Mozilla/5.0 (Windows NT 6.1; WOW64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/84.0.4147.125 Safari/537.36 OPR/70.0.3728.133 (Edition Yx)'
    }

    link = 'https://hh.ru/search/vacancy'
       
    html = requests.get(link, params=params, headers=headers)
    
    if html.ok:
        parsed_html = bs(html.text,'html.parser')
        
    
    for page in range(0, end_page):
        params['page'] = page
        html = requests.get(link, params=params, headers=headers)
        
        if html.ok:
            parsed_html = bs(html.text,'html.parser')
            
            vacancy_elements = parsed_html.find_all('div', {'class': 'vacancy-serp-item'})
                
            for element in vacancy_elements:
                vacancy_list.append(seach_hh_element(element))
                
    return vacancy_list

In [10]:
def seach_hh_element(element):

    vacancy_dict = {}
    
    vacancy_name = element.find('div', {'class': 'vacancy-serp-item__info'}).getText()
    
    vacancy_dict['vacancy_name'] = vacancy_name
    
    company_name = element.find('div', {'class': 'vacancy-serp-item__meta-info'}).find('a').getText()
    
    vacancy_dict['company_name'] = company_name
    
    city = element.find('span', {'class': 'vacancy-serp-item__meta-info'}).getText().split(', ')[0]
    
    vacancy_dict['city'] = city
    
    salary = element.find('div', {'class': 'vacancy-serp-item__sidebar'}).getText().replace('\xa0','')
    if salary == '':
        salary_min = None
        salary_max = None
        salary_currency = None
    else:
        salary = re.split(r' |-', salary)
        if salary[0] == 'до':
            salary_min = None
            salary_max = int(salary[1])
        elif salary[0] == 'от':
            salary_min = int(salary[1])
            salary_max = None
        else:
            salary_min = int(salary[0])
            salary_max = int(salary[1])            
        salary_currency = salary[2]
        
    vacancy_dict['salary_min'] = salary_min
    vacancy_dict['salary_max'] = salary_max
    vacancy_dict['salary_currency'] = salary_currency
    
    vacancy_link = element.find('span', {'class': 'resume-search-item__name'}).find('a')['href'].split('?')[0]
    
    vacancy_dict['vacancy_link'] = vacancy_link 
    
    vacancy_dict['site'] = 'hh.ru'
    
    return vacancy_dict

In [11]:
def seach_sjob(vacancy, end_page = 1):
    vacancy_list = []
    
    params = {
        'keywords': vacancy, \
        'geo%5Bt%5D%5B0%5D': '4', \
        'page': ''
        }
    
    headers = {
       'User-Agent': 'Mozilla/5.0 (Windows NT 6.1; WOW64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/84.0.4147.125 Safari/537.36 OPR/70.0.3728.133 (Edition Yx)'
    }
    
    link = 'https://www.superjob.ru/vacancy/search/'
        
    for page in range(0, end_page):
        params['page'] = page
        html = requests.get(link, params=params, headers=headers)
        
        if html.ok:
            parsed_html = bs(html.text,'html.parser')
            vacancy_elements = parsed_html.find_all('div', {'class': 'f-test-vacancy-item'})
                        
            for element in vacancy_elements:
                vacancy_list.append(seach_sjob_element(element))
                
    return vacancy_list

In [12]:
def seach_sjob_element(element):

    vacancy_dict = {}
    
    vacancy_name = item.find_all('a')[0].getText()
    vacancy_dict['vacancy_name'] = vacancy_name
    
    company_name = item.find('span', {'class': 'f-test-text-vacancy-item-company-name'}).getText()
    
    vacancy_dict['company_name'] = company_name
    
    company_city = item.find('span', {'class': 'f-test-text-company-item-location'}).findChildren()[1].getText().split(',')
    
    vacancy_dict['city'] = company_city[0]
    
    
    salary = item.find('span', {'class': 'f-test-text-company-item-salary'}).findChildren()[0].getText().replace(u'\xa0', u' ')
    
    if salary == 'По договорённости':
        salary_min = None
        salary_max = None
        salary_currency = None
    else:
        salary = re.split(r' |-', salary)
        if salary[0] == 'до':
            salary_min = None
            salary_max = int(salary[1]+salary[2])
            salary_currency = salary[3]
        elif salary[0] == 'от':
            salary_min = int(salary[1]+salary[2])
            salary_max = None
            salary_currency = salary[3]
        elif len(salary) == 5:
            salary_min = int(salary[0]+salary[1])
            salary_max = int(salary[3]+salary[4])            
            salary_currency = salary[4] 
        else:
            salary_min = int(salary[0]+salary[1])
            salary_max = None            
            salary_currency = salary[2]
            
    vacancy_dict['salary_min'] = salary_min
    vacancy_dict['salary_max'] = salary_max
    vacancy_dict['salary_currency'] = salary_currency
    
    vacancy_link = item.find_all('a')[0]['href']
    
    vacancy_dict['vacancy_link'] = f'https://www.superjob.ru{vacancy_link }'
    
    vacancy_dict['site'] = 'www.superjob.ru'
    
    return vacancy_dict

In [13]:
vacancy = 'Data Scientist'
lenght = 5

vacancy_dict = []
vacancy_dict.extend(seach_hh(vacancy, lenght))
vacancy_dict.extend(seach_sjob(vacancy, lenght))

df = pd.DataFrame(vacancy_dict)
    

In [14]:
df

,vacancy_name,company_name,city,salary_min,salary_max,salary_currency,vacancy_link,site
0,Специалист по углубленной аналитике данных (Da...,"А1. Информационные технологии, телеком",Минск,NaN,NaN,None,https://tula.hh.ru/vacancy/37732722,hh.ru
1,Data Scientist,ООО КИБЕРБОРЕЯ,Москва,120000.0,170000.0,руб.,https://tula.hh.ru/vacancy/38660521,hh.ru
2,Data scientist (middle/junior),Открытые Технологии,Москва,NaN,120000.0,руб.,https://tula.hh.ru/vacancy/38961658,hh.ru
3,Программист Python/ Data Scientist (junior/mid...,ООО Апэрбот,Новосибирск,70000.0,150000.0,руб.,https://tula.hh.ru/vacancy/38618845,hh.ru
4,Data Scientist,GFAIVE,Москва,150000.0,NaN,руб.,https://tula.hh.ru/vacancy/38883862,hh.ru
...,...,...,...,...,...,...,...,...
262,Senior Data Scientist,Nasctech,Киев,NaN,NaN,None,https://tula.hh.ru/vacancy/38617269,hh.ru
263,Data Scientist,Тинькофф,Москва,NaN,NaN,None,https://tula.hh.ru/vacancy/37559909,hh.ru
264,Младший специалист Data Scientist,Северсталь. ИТ Компания,Москва,NaN,NaN,None,https://tula.hh.ru/vacancy/38852806,hh.ru
265,Стажер data scientist,Сибур,Москва,NaN,NaN,None,https://tula.hh.ru/vacancy/38502536,hh.ru


# Урок 3. Системы управления базами данных MongoDB и SQLite в Python

In [5]:
from pymongo import MongoClient

In [57]:
def add_in_mongodb(vacancy_dict):
    mongodb = MongoClient('localhost', 27017)
    db = mongodb['vacancy_db']
    collection = db['vacancy']
    
    for vacancy_one in vacancy_dict:
        vacancy_name = vacancy_one['vacancy_name']
        company_name = vacancy_one['company_name']
        city = vacancy_one['city']
        salary_min = vacancy_one['salary_min']
        salary_max = vacancy_one['salary_max']
        salary_currency = vacancy_one['salary_currency']
        vacancy_link = vacancy_one['vacancy_link']
        site = vacancy_one['site']
        log = len(list(db.collection.find({'vacancy_name': vacancy_name, 'company_name': company_name, 'city': city,
                                       'salary_min': salary_min,'salary_max': salary_max,'salary_currency': salary_currency
                                       ,'vacancy_link': vacancy_link, 'site': site })))
        if  log == 0:
            db.collection.insert_one(vacancy_one)
        
    return "Done"    

In [58]:
add_in_mongodb(vacancy_dict)

'Done'

In [7]:
def seach_salary(salary):
    mongodb = MongoClient('localhost', 27017)
    db = mongodb['vacancy_db']
    collection = db['vacancy']
    
    for vacancy in db.collection.find( { '$or': [ { 'salary_min': { '$gt': salary } }, { 'salary_max': { '$gt': salary } } ] } ):
        pprint(vacancy)
    
    return "Done" 

In [12]:
seach_salary(200000)

{'_id': ObjectId('5f538b8ca0db748cb49de3dc'),
 'city': 'Москва',
 'company_name': 'ООО РАБЛЗ',
 'salary_currency': 'руб.',
 'salary_max': 500000,
 'salary_min': None,
 'site': 'hh.ru',
 'vacancy_link': 'https://tula.hh.ru/vacancy/38576646',
 'vacancy_name': 'Data Scientist (middle or senior)'}
{'_id': ObjectId('5f538b8ca0db748cb49de45a'),
 'city': 'Москва',
 'company_name': ' Agiloft',
 'salary_currency': 'руб.',
 'salary_max': 280000,
 'salary_min': 200000,
 'site': 'hh.ru',
 'vacancy_link': 'https://tula.hh.ru/vacancy/38744066',
 'vacancy_name': 'Senior Data Scientist (remote)'}
{'_id': ObjectId('5f538b8ca0db748cb49de460'),
 'city': 'Томск',
 'company_name': ' Agiloft',
 'salary_currency': 'руб.',
 'salary_max': 280000,
 'salary_min': 200000,
 'site': 'hh.ru',
 'vacancy_link': 'https://tula.hh.ru/vacancy/38744084',
 'vacancy_name': 'Senior Data Scientist (remote)'}
{'_id': ObjectId('5f538b8ca0db748cb49de468'),
 'city': 'Новосибирск',
 'company_name': ' Agiloft',
 'salary_currency': '

'Done'